In [3]:
from numpy import linspace, meshgrid, array, sin, cos, pi, abs
from mayavi import mlab

import matplotlib.pyplot as plt
import matplotlib.image as mpimg



In [4]:

def local_plot(mlab_popup=None,mlab_output=None):

    mlab_popup  = (False if mlab_popup is None else mlab_popup)
    mlab_output = ('popup.png' if mlab_output is None else mlab_popup)

    if mlab_popup:
        mlab.show()
    else:
        # mlab.figure(size=(700, 830))
        mlab.figure(size=(1400, 1660))
        print('savefig: ',mlab_output)
        mlab.savefig(mlab_output)
        img=mpimg.imread('example.png')
        imgplot = plt.imshow(img)
        plt.show()
        
    return True


In [5]:
import mpl_toolkits.mplot3d as a3

class stare_prism(object):
    
    def __init__(self,siv=None,tiv=None,color_reverse=None,color=None,color_forward=None,tiv_mock=None):
        self.siv = (None if siv is None else siv)
        self.tiv = (None if tiv is None else tiv)
        self.tiv_mock = ([0.0,0.33,0.67,1.0] if tiv_mock is None else tiv_mock)
        self.lons, self.lats, self.intmat = pystare.triangulate_indices([self.siv])
        
        if color_reverse is None and color is None and color_forward is None:
            self.color_reverse = 'red'
            self.color         = 'green'
            self.color_forward = 'blue'
        elif color is not None:
            self.color_reverse = (color if color_reverse is None else color_reverse)
            self.color         =  color
            self.color_forward = (color if color_forward is None else color_forward)
        else:
            self.color_reverse = ('red'   if color_reverse is None else color_reverse)
            self.color         = 'green'
            self.color_forward = ('blue'  if color_forward is None else color_forward)            
        
        return
    
    def plot1_simple(
        self
              ,figax
              ,alpha      = None
              ,edge_alpha = None
              ,edge_color = None
              ,prism_edge_color = None
              ,end_faces_plot = None
              ,side_face_edges_plot = None
             ):
        
        alpha               = (0.5 if alpha is None else alpha)
        edge_alpha          = (1 if edge_alpha is None else edge_alpha)
        end_faces_plot = ([True]*6 if end_faces_plot is None else end_faces_plot)
        side_face_edges_plot = ([True]*3 if side_face_edges_plot is None else side_face_edges_plot)
        
        self.plot0(figax
                   ,color = self.color
                   ,edge_color = edge_color
                   ,prism_edge_color = prism_edge_color
                   ,alpha      = alpha
                   ,edge_alpha = edge_alpha                   
                   ,z=[self.tiv_mock[0],self.tiv_mock[-1]]
                   ,end_faces_plot = [end_faces_plot[0],end_faces_plot[-1]]
                   ,side_face_edges_plot = side_face_edges_plot
                  )

        return figax
    
    def plot1(self
              ,figax
              ,alpha      = None
              ,edge_alpha = None
              ,edge_color = None
              ,prism_edge_color = None
              ,end_faces_plot = None
              ,side_face_edges_plot = None
             ):
        
        alpha               = (0.5 if alpha is None else alpha)
        edge_alpha          = (0.5 if edge_alpha is None else alpha)
        side_face_edges_plot = ([True]*3 if side_face_edges_plot is None else side_face_edges_plot)
        
        end_faces_plot = ([True]*6 if end_faces_plot is None else end_faces_plot)
        
        self.plot0(figax
                   ,color      = self.color_reverse
                   ,edge_color = edge_color
                   ,prism_edge_color = prism_edge_color
                   ,alpha      = alpha
                   ,edge_alpha = edge_alpha
                   ,z=self.tiv_mock[0:2]
                   ,end_faces_plot = end_faces_plot[0:2]
                   ,side_face_edges_plot = side_face_edges_plot
                  )
        self.plot0(figax
                   ,color = self.color
                   ,edge_color = edge_color
                   ,prism_edge_color = prism_edge_color
                   ,alpha      = alpha
                   ,edge_alpha = edge_alpha
                   ,z=self.tiv_mock[1:3]
                   ,end_faces_plot = end_faces_plot[2:4]
                   ,side_face_edges_plot = side_face_edges_plot
                  )
        self.plot0(figax
                   ,color = self.color_forward
                   ,edge_color = edge_color
                   ,prism_edge_color = prism_edge_color
                   ,alpha      = alpha
                   ,edge_alpha = edge_alpha
                   ,z=self.tiv_mock[2:4]
                   ,end_faces_plot = end_faces_plot[4:6]
                   ,side_face_edges_plot = side_face_edges_plot
                  )
        
        return figax
    
    def plot0(self
             ,figax
             ,z=None
             ,color=None
             ,edge_color=None
             ,prism_edge_color=None
             ,alpha=None
             ,edge_alpha=None
             ,end_faces_plot = None
             ,side_face_edges_plot = None
             ,dbg=None
            ):
        
        z          = ([0.0,1.0] if z is None else z)
        color      = ('grey' if color is None else color)
        edge_color = (None if edge_color is None else edge_color)
        prism_edge_color = (edge_color if prism_edge_color is None else prism_edge_color)
        alpha      = (0.5 if alpha is None else alpha)
        edge_alpha = (1.0 if edge_alpha is None else edge_alpha)
        end_faces_plot = ([True,True] if end_faces_plot is None else end_faces_plot)
        side_face_edges_plot = ([True]*3 if side_face_edges_plot is None else side_face_edges_plot)
        dbg        = (False if dbg is None else dbg)
        
        # z0=self.tiv[0] # bad
        # temporal_scales = [self.tiv[1]]*4
        
        # print('side face edges plot: ',side_face_edges_plot)
        
        z0_ = z[0]
        z1_ = [z[1]]
        
        for tr in self.intmat:
            
            if dbg:
                print('tr: ',tr)
            if z1_ is None:
                z = [z0_]*4
            else:
                z = [z0_]*4
                z1 = numpy.array([z1_[int(tr[0]/3)]
                                ,z1_[int(tr[1]/3)]
                                ,z1_[int(tr[2]/3)]
                                ,z1_[int(tr[0]/3)]])
            x = make_cyclic(numpy.array([self.lons[tr[0]],self.lons[tr[1]],self.lons[tr[2]]]))
            y = make_cyclic(numpy.array([self.lats[tr[0]],self.lats[tr[1]],self.lats[tr[2]]]))

            if dbg:
                print('==')
                print(sgn(x,y))
        
            thr = 180
        
            if sgn(x,y) < 0:
                if True:
                    if abs(x[0]-x[1]) > thr:
                        if x[0] < x[1]:
                            x[0] += 360
                            x[3] = x[0]
                        else:
                            x[1] += 360
                if True:
                    if abs(x[0]-x[2]) > thr:
                        if x[0] < x[2]:
                            x[0] += 360
                            x[3] = x[0]
                        else:
                            x[2] += 360            
                if True:
                    if abs(x[1]-x[2]) > thr:
                        if x[1] < x[2]:
                            x[1] += 360.0
                        else:
                            x[2] += 360.0
                        
            if dbg:
                print(x)
                print(y)
                print(sgn(x,y))
                print('.')
                
                print('end_faces_plot: ',end_faces_plot)
    
            x=numpy.array(x)
            y=numpy.array(y)
            z=numpy.array(z)
    
            if end_faces_plot[0]:
                figax.ax.plot_trisurf(x,y,z
                                    ,color=color
                                    ,alpha=alpha
                                    )
            else:
                # This is stupid
                 figax.ax.plot_trisurf(x,y,z
                                    ,color=color
                                    ,alpha=0
                                    )               
        
            if z1 is not None:
                
                if dbg:
                    print(100)
                    
                if prism_edge_color is not None:
                    for i in [0,1,2]:
                        figax.ax.plot3D([x[i],x[i]],[y[i],y[i]],[z[i],z1[i]]
                                        ,c=prism_edge_color
                                        ,alpha=edge_alpha
                                       )                
                
                for i in [0,1,2]:
                    if True:
                    
                        fx = numpy.array( [x[i],x[i+1],x[i+1]] )
                        fy = numpy.array( [y[i],y[i+1],y[i+1]] )
                        fz = numpy.array( [z[i],z[i+1],z1[i]] )
                        
                        vtx = numpy.array(
                            [
                                [x[i],x[i+1],x[i+1]]
                                ,[y[i],y[i+1],y[i+1]]
                                ,[z[i],z[i+1],z1[i]]
                            ]
                        )
                        vtx = numpy.transpose(vtx)
                        if dbg:
                            print('vtx: ',vtx)
                        tri = a3.art3d.Poly3DCollection([vtx])
                        tri.set_facecolor(
                            color
                            # self.color_reverse
                            # mpl.colors.rgb2hex([0.5,0.5,0.5])
                        )
                        tri.set_alpha(alpha)
                        if side_face_edges_plot[i]:
                            tri.set_edgecolor(edge_color)
                        else:
                            tri.set_edgecolor(None)
                        figax.ax.add_collection3d(tri)
     
                        vtx = numpy.array(
                            [
                                [x[i],x[i+1],x[i]]
                                ,[y[i],y[i+1],y[i]]
                                ,[z[i],z1[i+1],z1[i]]
                            ]
                        )
                        vtx = numpy.transpose(vtx)
                        tri = a3.art3d.Poly3DCollection([vtx])
                        tri.set_color(color)
                        # tri.set_color(mpl.colors.rgb2hex([0.45,0.45,0.45]))
                        tri.set_alpha(alpha)
                        if side_face_edges_plot[i]:
                            tri.set_edgecolor(edge_color)
                        else:
                            tri.set_edgecolor(None)
                        figax.ax.add_collection3d(tri)
                    
            if end_faces_plot[0]:
                if prism_edge_color is not None:
                    figax.ax.plot3D(x, y, z
                        ,c=prism_edge_color
                        ,alpha=edge_alpha
                        ) 
                
            if end_faces_plot[1]:
                if prism_edge_color is not None:                    
                    figax.ax.plot3D(x, y, z1, c=prism_edge_color, alpha=edge_alpha) 
                figax.ax.plot_trisurf(x,y,z1,color=color,alpha=alpha)

        return figax
        